Preparing dataset for analysis.

Outliers are considerably higher or lower than the rest of the data. Investigate further for data cleaning.

Not all outliers are bad data points

In [ ]:
df.shape() # to see the structure
df.info() # to get more information about a database
df.describe() # shows more information about the database

df.continent.value_counts(dropna=False) # perform a frequency count
df.continent.value_counts(dropna=False).haed() # returns top categories
# for example each country should have 1 observation but sweden has 2

Data visualization helps point out outliers.

Bar plots for discrete data counts

Histograms for continous data

Box plots visualize basic summary statistics

Boxes show on the plot, the lines that extend from it are called whiskers. Outliers are values that are beyond the whiskers.

In [ ]:
df[df.population > 1000000000] # find countries with over 1 billion people

# Import matplotlib.pyplot
import matplotlib.pyplot as plt

# Plot the histogram
df['Existing Zoning Sqft'].plot(kind='hist', rot=70, logx=True, logy=True)

# Display the histogram
plt.show()

# Create the boxplot
df.boxplot(column='initial_cost', by='Borough', rot=90)

# Create and display the first scatter plot
df.plot(kind='scatter', x='initial_cost', y='total_est_fee', rot=70)

You can see the 2 extreme outliers are in the borough of Manhattan. An initial guess could be that since land in Manhattan is extremely expensive, these outliers may be valid data points. Again, further investigation is needed to determine whether or not you can drop or keep those points in your data.

When you want to visualize two numeric columns, scatter plots are ideal.

In general, from the second plot it seems like there is a strong correlation between 'initial_cost' and 'total_est_fee'. In addition, take note of the large number of points that have an 'initial_cost' of 0. It is difficult to infer any trends from the first plot because it is dominated by the outliers.

Tidy data provides a standard way to organize data values within a dataset - "Tidy Data" by Hadley Wickham

3 principles of tidy data

* columns represent separate variables

* rows represent individual observations

* observational units form tables

some data is better for reporting and some is better for analysis. Tidy data makes it easier to fix common data problems.

The problem is columns containing data values instead of variables.

Fix this by using pd.melt()

id_vars is a fixed column

value_vars is which columns we want to melt

var_name, value_name parameters to name the column names

In [ ]:
airquality_melt = pd.melt(airquality, id_vars=['Month', 'Day']) # don't want to melt id_vars
# rename columns
airquality_melt = pd.melt(airquality, id_vars=['Month', 'Day'], var_name='measurement', value_name='reading')

Pivot is the opposite of melting

Useful to turn analysis friendly to report friendly or when rows contain observations.

Pivots can't work with duplicate entries. Can use Pivot Table instead.

In [ ]:
airquality_pivot = airquality_melt.pivot_table(index=['Month', 'Day'], columns='measurement', values='reading')

# Reset the index of airquality_pivot: airquality_pivot
airquality_pivot = airquality_pivot.reset_index() # get back to original df

Melting and pivoting are the basic tools. Another problem is when columns contain multiple bits of information.

For example, want a separate sex and age group column instead of a combined column.

1st melt data down

2nd parse out sex and age column

In [ ]:
# Melt tb: tb_melt
tb_melt = pd.melt(tb, id_vars=['country', 'year'])

# Create the 'gender' column
tb_melt['gender'] = tb_melt.variable.str[0]

# Create the 'age_group' column
tb_melt['age_group'] = tb_melt.variable.str[1:]

# Print the head of tb_melt
print(tb_melt.head())

In [ ]:
# Melt ebola: ebola_melt
ebola_melt = pd.melt(ebola, id_vars=['Date', 'Day'], var_name='type_country', value_name='counts')

# Create the 'str_split' column
ebola_melt['str_split'] = ebola_melt.type_country.str.split('_')

# Create the 'type' column
ebola_melt['type'] = ebola_melt.str_split.str.get(0)

# Create the 'country' column
ebola_melt['country'] = ebola_melt.str_split.str.get(1)

Data may not always come in 1 huge file.

Need to combine all the data.

Concatenate two dataframes into a single dataframe. pd.concat(), don't forget to reset the index label ignore_index=True

In [ ]:
# Concatenate uber1, uber2, and uber3: row_concat
row_concat = pd.concat([uber1, uber2, uber3])

axis=0 # row-wise concatenation
axis=1 # col-wise concatenation
ignore_index=True

In order to concatenate dfs they must be in a list.

What happens if there are thousands of files? glob function

globbing - pattern matching for file names

*.csv matches any csv

file_?.csv matches any single character

this returns the list of file names

add the dataframes into a list to concatenate multiple files at once

In [ ]:
# Import necessary modules
import pandas as pd
import glob

# Write the pattern: pattern
pattern = '*.csv'

# Save all file matches: csv_files
csv_files = glob.glob(pattern)

# Print the file names
print(csv_files)

# Load the second file into a DataFrame: csv2
csv2 = pd.read_csv(csv_files[1])

In [ ]:
# Create an empty list: frames
frames = []

#  Iterate over csv_files
for csv in csv_files:

    #  Read csv into a DataFrame: df
    df = pd.read_csv(csv)
    
    # Append df to frames
    frames.append(df)

# Concatenate frames into a single DataFrame: uber
uber = pd.concat(frames)

combine data (ex. organization of the data is not the same)

can merge data instead, like based on column names

* one-to-one 

* one-to-many where columns are populated with other tables data

* many-to-many

all use the same function

Merge the site and visited DataFrames on the 'name' column of site and 'site' column of visited.

In [ ]:
# Merge the DataFrames: o2o
o2o = pd.merge(left=site, right=visited, left_on='name', right_on='site')

# Merge site and visited: m2m
m2m = pd.merge(left=site, right=visited, left_on='name', right_on='site')

# Merge m2m and survey: m2m
m2m = pd.merge(left=m2m, right=survey, left_on='ident', right_on='taken')

df.dtypes

object dtype is typically a string

numeric cols can be strings or vice versa

converting data to category

* can make df smaller in memory

* can make them utilized by other Python libraries for analysis

if data should be float but come out as string, then bad data

errors='coerce' means to convert values into a numeric value, invalid values will be set as NaN

In [ ]:
# Convert the sex column to type 'category'
tips.sex = tips.sex.astype('category')

# Convert 'total_bill' to a numeric dtype
tips['total_bill'] = pd.to_numeric(tips['total_bill'], errors='coerce')

# Print the info of tips
print(tips.info())

much of data cleaning involves string manipulation because the world's data is unstructured text

also need to do string manipulation to make them consistent ex. 17, $17, $17.89

re library for regular expressions for string pattern matching

$ means to match from the end of the string so need to \$\d*

/d*

$17.00 /$/d*/./d*

$17.89 /$/d*/./d{2}

$17.895 ^\$\d*\.\d{2}$ where ^ matches the beginning and $ matches end, exact regex matching

compile the pattern then use the pattern to match values like matching a value down pd columns

In [ ]:
# Import the regular expression module
import re

# Compile the pattern: prog
prog = re.compile('\d{3}-\d{3}-\d{4}')

# See if the pattern matches
result = prog.match('123-456-7890')
print(bool(result))

# See if the pattern matches
result = prog.match('1123-456-7890')
print(bool(result))

\d is the pattern required to find digits. This should be followed with a + so that the previous element is matched one or more times. This ensures that 10 is viewed as one number and not as 1 and 0.

In [ ]:
# Find the numeric values: matches
matches = re.findall('\d+', 'the recipe calls for 10 strawberries and 1 banana')

# Print the matches
print(matches)

In [ ]:
# Write the first pattern
pattern1 = bool(re.match(pattern='\d{3}-\d{3}-\d{4}', string='123-456-7890'))
print(pattern1)

# Write the second pattern
pattern2 = bool(re.match(pattern='\$\d*\.\d{2}', string='$123.45'))
print(pattern2)

# Write the third pattern
pattern3 = bool(re.match(pattern='\w*', string='Australia'))
print(pattern3)

complex cleaning sometimes requires multiple steps, can use a python function with .apply

In [ ]:
# Define recode_sex()
def recode_sex(sex_value):

    # Return 1 if sex_value is 'Male'
    if sex_value == 'Male':
        return 1
    
    # Return 0 if sex_value is 'Female'    
    elif sex_value == 'Female':
        return 0
    
    # Return np.nan    
    else:
        return np.nan

# Apply the function to the sex column
tips['sex_recode'] = tips.sex.apply(recode_sex)

# Print the first five rows of tips
print(tips.head())

In [ ]:
# Write the lambda function using replace
tips['total_dollar_replace'] = tips.total_dollar.apply(lambda x: x.replace('$', ''))

# Write the lambda function using regular expressions
tips['total_dollar_re'] = tips.total_dollar.apply(lambda x: re.findall('\d+\.\d+', x)[0])

# Print the head of tips
print(tips.head())

duplicate data skews results

.drop_duplicates() to address

missing data

* leave as-is

* drop them

* fill missing value

tips_nan.info() to get a count of missing value

tips_nan.dropna() drops nans but you can lose too much data with this method

you can also drop columns

.fillna() - can fill with provided value ex. missing or 0, can fill with test statistic like mean, make sure the value makes sense, median is a better statistic to find the presence of outliers

In [ ]:
# Create the new DataFrame: tracks
tracks = billboard[['year', 'artist', 'track', 'time']]

# Print info of tracks
print(tracks.info())

# Drop the duplicates: tracks_no_duplicates
tracks_no_duplicates = tracks.drop_duplicates()

# Print info of tracks
print(tracks_no_duplicates.info())

In [ ]:
# Calculate the mean of the Ozone column: oz_mean
oz_mean = airquality.Ozone.mean()

# Replace all the missing values in the Ozone column with the mean
airquality['Ozone'] = airquality.Ozone.fillna(oz_mean)

# Print the info of airquality
print(airquality.info())

Assert statements check programmatically vs visually

if we drop or fill NaNs, expect 0 missing values

can write an assert statement to verify

detect early warnings and errors

true assert statements return nothing, false assert statements return an error

The first .all() method will return a True or False for each column, while the second .all() method will return a single True or False.

In [ ]:
# Assert that there are no missing values
assert pd.notnull(ebola).all().all()

# Assert that all values are >= 0
assert (ebola >= 0).all().all()

In [ ]:
# Import matplotlib.pyplot
import matplotlib.pyplot as plt

# Create the scatter plot
g1800s.plot(kind='scatter', x='1800', y='1899')

# Specify axis labels
plt.xlabel('Life Expectancy by Country in 1800')
plt.ylabel('Life Expectancy by Country in 1899')

# Specify axis limits
plt.xlim(20, 55)
plt.ylim(20, 55)

# Display the plot
plt.show()

In [ ]:
def check_null_or_valid(row_data):
    """Function that takes a row of data,
    drops all missing values,
    and checks if all remaining values are greater than or equal to 0
    """
    no_na = row_data.dropna()[1:-1]
    numeric = pd.to_numeric(no_na)
    ge0 = numeric >= 0
    return ge0

# Check whether the first column is 'Life expectancy'
assert g1800s.columns[0] == 'Life expectancy'

# Check whether the values in the row are valid
assert g1800s.iloc[:, 1:].apply(check_null_or_valid, axis=1).all().all()

# Check that there is only one instance of each country
assert g1800s['Life expectancy'].value_counts()[0] == 1

In [ ]:
# Melt gapminder: gapminder_melt
gapminder_melt = pd.melt(gapminder, id_vars='Life expectancy')

# Rename the columns
gapminder_melt.columns = ['country', 'year', 'life_expectancy']

# Print the head of gapminder_melt
print(gapminder_melt.head())

In [ ]:
# Convert the year column to numeric
gapminder.year = pd.to_numeric(gapminder.year)

# Test if country is of type object
assert gapminder.country.dtypes == np.object

# Test if year is of type int64
assert gapminder.year.dtypes == np.int64

# Test if life_expectancy is of type float64
assert gapminder.life_expectancy.dtypes == np.float64

In [ ]:
# Create the series of countries: countries
countries = gapminder['country']

# Drop all the duplicates from countries
countries = countries.drop_duplicates()

# Write the regular expression: pattern
pattern = '^[A-Za-z\.\s]*$'

# Create the Boolean vector: mask
mask = countries.str.contains(pattern)

# Invert the mask: mask_inverse
# Invert the mask by placing a ~ before it.
mask_inverse = ~mask

# Subset countries using mask_inverse: invalid_countries
invalid_countries = countries.loc[mask_inverse]

# Print invalid_countries
print(invalid_countries)

In [ ]:
# Assert that country does not contain any missing values
assert pd.notnull(gapminder.country).all()

# Assert that year does not contain any missing values
assert pd.notnull(gapminder.year).all()

# Drop the missing values
gapminder = gapminder.dropna(axis=0, how='any')

# Print the shape of gapminder
print(gapminder.shape)

In [ ]:
# Add first subplot
plt.subplot(2, 1, 1) 

# Create a histogram of life_expectancy
gapminder.life_expectancy.plot(kind='hist')

# Use the .groupby() method on gapminder with 'year' as the argument. 
# Then select 'life_expectancy' and chain the .mean() method to it.
# Group gapminder: gapminder_agg
gapminder_agg = gapminder.groupby('year')['life_expectancy'].mean()

# Print the head of gapminder_agg
print(gapminder_agg.head())

# Print the tail of gapminder_agg
print(gapminder_agg.tail())

# Add second subplot
plt.subplot(2, 1, 2)

# Create a line plot of life expectancy per year
gapminder_agg.plot()

# Add title and specify axis labels
plt.title('Life expectancy over the years')
plt.ylabel('Life expectancy')
plt.xlabel('Year')

# Display the plots
plt.tight_layout()
plt.show()

# Save both DataFrames to csv files
gapminder.to_csv('gapminder.csv')
gapminder_agg.to_csv('gapminder_agg.csv')